<a href="https://colab.research.google.com/github/jrakhshanda/Bayesian-Methods/blob/master/Topic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyLDAvis

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import wordnet

import regex as re
from nltk.stem.porter import PorterStemmer 
from nltk.tokenize import word_tokenize, sent_tokenize,RegexpTokenizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

import pickle
import os
from os.path import isfile, join

#for vectorizing
nltk.download('stopwords')
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

import gensim.corpora as corpora
from pprint import pprint
import gensim

import spacy

import pyLDAvis.gensim
import pickle 
import pyLDAvis

#comment this out if you're not able to view any of the print lines
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%config InlineBackend.figure_format = 'retina'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#Data

In [2]:
df = pd.read_csv('/content/drive/MyDrive/data/data.zip', compression='zip', header=0)
df = df.loc[:,~df.columns.str.match("Unnamed")]
df.head()

,PMID,Title,Abstract
0,33581499,"PCNA, a focus on replication stress and the al...","The maintenance of telomeres, which are specia..."
1,33581339,Compositional Variability and Mutation Spectra...,COVID-19 and its causative pathogen SARS-CoV-2...
2,33580188,Histone variant H2A.B-H2B dimers are spontaneo...,H2A.B is an evolutionarily distant histone H2A...
3,33580181,Hypermutated phenotype in gliosarcoma of the s...,Gliosarcoma is a variant of glioblastoma with ...
4,33580144,Arsenic hexoxide has differential effects on c...,Arsenic is reportedly a biphasic inorganic com...


In [3]:
df['Text'] = df['Title'] + '. ' + df['Abstract']
df['Text'] = df['Text'].astype('str')

# Reuseable Routiens

In [4]:
def save_object(path, filename, obj):
    print('Saving Object')
    path_file = join(path, filename)    
    pickle.dump(obj, open(path_file, 'wb'))
    print('Save complete')

def load_object(path, filename):
    print('Attempting to Load Object')
    path_file = join(path, filename)    
    obj = pickle.load(open(path_file, "rb" ))
    print('Load complete')
    return obj


def save_csv(path, filename, dataframe):
    print('Saving Dataframe to CSV')
    path_file = join(path, filename)    
    dataframe.to_csv(path_file, index=False)

def load_csv(path, filename):
    print('Loading Dataframe From CSV')
    path_file = join(path, filename)
    dataframe = pd.read_csv(path_file)
    return dataframe

# Pre-processing

In [28]:
stopwords = nltk.corpus.stopwords.words('english')
newStopWords = ['type','wild','function','observed','results','different','identified','from','furthermore','process','model','effect','known','suggest','damage','human','rat','rats',
                'suggested','conclusion','determined','indicate','Moreover','system', 'form', 'treatment','disease','although','conclusion','including','exposure','repair','injury','cell',
                'functions','compared','interaction','level','demonstrated','studies','substrate','interaction','shown','contrast','presence','important', 'response','cells', 'effect','risk',
                'induction', 'increase', 'also', 'breaks','treated','showed','used','comet', 'affect', 'may', 'two','study','gene','end','ends','let','double','single','strands','stress',
                'pathway','role','demonstrate','present','result','use','occur','appear','joining','affect','cause','induce','defect','induced','lead','complex','resistance','calls','ber',
                'analyzed','include','similar','studied','method','addition','suggesting','background','gene','expression','investigated','promote','types','dna','proteins','protein', 'ner',
                'high','low','agents','using','strands','cycle','drug','investigate','increased','that','induce','structure','bindings','binding','molecular','mouse','mice','activity',
                'levels','level','patient','patients','dose','drug','recombination','yeast','strand','mutants','mutations','replications','homologous','genetic','genes','associative','lung',
                'specific','light','molecular','binding','sequence','lines','cancer','poly','inhibitation','activation','nuclear','regulation','tumor','mismatch','excision','base',
                'cellular','growth','irridation','tumors','mutations','transription','resistance','site','adducts','dose','enzyme','associated','analysis','species','tissue',
                'factor','domain','clinical','cancers','liver','assay','irradiated','irradiation','nucleotide','one','damage','increase','resist','mechanism','association','group','groups','aging']
stopwords.extend(newStopWords)

In [29]:
def preprocess(text):
  text = stemmer.stem(text) 
  tokenizer = RegexpTokenizer(r'\w+')
  words_tokens = tokenizer.tokenize(text) 
  tokens = []  
  for w in words_tokens:  
    if w not in stopwords:  
      tokens.append(w)  
  tokens = [x for x in tokens if len(x) >= 3]
  return tokens

In [30]:
tokens = [preprocess(i) for i in df['Text']]

In [37]:
save_object('/content/drive/MyDrive/models','topic_tokens',tokens)

Saving Object
Save complete


# LDA model Training

In [38]:
# Create Dictionary
id2word = corpora.Dictionary(tokens)
# Create Corpus
texts = tokens
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 5), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


In [ ]:
# number of topics
num_topics = 8
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


In [41]:
pprint(lda_model.print_topics())

[(0,
  '0.011*"mutation" + 0.007*"syndrome" + 0.006*"breast" + 0.006*"colorectal" + '
  '0.005*"atm" + 0.004*"hereditary" + 0.004*"cases" + 0.004*"hnpcc" + '
  '0.004*"found" + 0.004*"polymorphisms"'),
 (1,
  '0.004*"telomere" + 0.004*"mitochondrial" + 0.003*"beta" + 0.003*"wound" + '
  '0.003*"normal" + 0.003*"age" + 0.003*"control" + 0.003*"tgf" + 0.002*"bone" '
  '+ 0.002*"significantly"'),
 (2,
  '0.014*"p53" + 0.008*"apoptosis" + 0.006*"effects" + 0.006*"inhibition" + '
  '0.005*"radiation" + 0.005*"parp" + 0.005*"inhibitors" + 0.005*"mechanisms" '
  '+ 0.004*"kinase" + 0.004*"dependent"'),
 (3,
  '0.006*"chromatin" + 0.005*"transcription" + 0.005*"coli" + '
  '0.004*"replication" + 0.004*"strain" + 0.004*"genome" + 0.003*"involved" + '
  '0.003*"strains" + 0.003*"mutant" + 0.003*"dependent"'),
 (4,
  '0.006*"replication" + 0.005*"coli" + 0.004*"synthesis" + 0.004*"lesions" + '
  '0.004*"reca" + 0.003*"polymerase" + 0.003*"mutant" + 0.003*"escherichia" + '
  '0.003*"oxidative" + 0

## Analysis of Results

### What is the Dominant topic and its percentage contribution in each document?
In LDA models, each document is composed of multiple topics. But, typically only one of the topics is dominant. The below code extracts this dominant topic for each sentence and shows the weight of the topic and the keywords in a nicely formatted output.

This way, you will know which document belongs predominantly to which topic.

In [48]:
def format_topics_sentences(ldamodel=None, corpus = corpus, texts= texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return sent_topics_df


In [49]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=texts)

In [61]:
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['PMID', 'Dominant Topic', 'Topic % Contribution', 'Keywords', 'Text']
df_dominant_topic['PMID'] = df['PMID']
df_dominant_topic.head(8)
df_dominant_topic.to_csv('dominant_topics.csv')
#save_csv('/content/drive/MyDrive/data','dominant_topics',df_dominant_topic)

,PMID,Dominant Topic,Topic % Contribution,Keywords,Text
0,33581499,1.0,0.4045,"telomere, mitochondrial, beta, wound, normal, ...","[pcna, focus, replication, alternative, length..."
1,33581339,3.0,0.3878,"chromatin, transcription, coli, replication, s...","[compositional, variability, mutation, spectra..."
2,33580188,3.0,0.9912,"chromatin, transcription, coli, replication, s...","[histone, variant, h2a, h2b, dimers, spontaneo..."
3,33580181,6.0,0.3826,"cisplatin, survival, radiation, chemotherapy, ...","[hypermutated, phenotype, gliosarcoma, spinal,..."
4,33580144,2.0,0.6327,"p53, apoptosis, effects, inhibition, radiation...","[arsenic, hexoxide, differential, effects, pro..."
5,33579976,6.0,0.4223,"cisplatin, survival, radiation, chemotherapy, ...","[persistent, stag2, mutation, despite, multimo..."
6,33579974,0.0,0.5129,"mutation, syndrome, breast, colorectal, atm, h...","[somatic, aberrations, benign, breast, subsequ..."
7,33579938,5.0,0.7292,"genome, methylation, mgmt, mutation, crispr, m...","[whole, genome, sequencing, data, derive, defi..."


Saving Dataframe to CSV


In [ ]:
unique = set(df_dominant_topic['Keywords'].str.split(' ').sum())

print(list(sorted(unique)))

### The most representative sentence for each topic

In [ ]:
# Display setting to show more characters in column
pd.options.display.max_colwidth = 100

sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf_mallet.head(10)

### Visualize The topics

In [45]:
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, '/content/drive/MyDrive/models'+ str(num_topics) +'.html')
LDAvis_prepared

FileNotFoundError: ignored